# Makam pitch sequence classification with LSTM

### Library importing for file reading and preprocessing

### Preprocessing pitch files

Before proceeding, the pitch files on the CompMusic Dunya makam corpus need to be converted in the quantized pitch series encoding constructed as described in the pseudocode below.

In [1]:
import glob
import os
import numpy as np
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


### File reading, octave folded encoding

In [2]:
#quantized octave-folded pitch file directory
ofq_read_dir = "./octfold_qdata/" 

#Makam list for more efficient file searching during label retrieval
makams = ["Acemasiran", "Acemkurdi", "Bestenigar", "Beyati", "Hicaz", "Hicazkar", "Huseyni", "Huzzam", "Karcigar", "Kurdilihicazkar", "Mahur", "Muhayyer", "Neva", "Nihavent", "Rast", "Saba", "Segah", "Sultaniyegah", "Suzinak", "Ussak"]

all_qpitch = [] #array holding all quantized pitch series as strings
y = [] #holds makam labels
max_length = 0
for root, dirs, files in os.walk(ofq_read_dir):
    for name in files:
        if '.pitch' in name:
            #retrieve label from parent of original path
            for makam in makams:
                if (os.path.isfile("./otmm_makam_recognition_dataset/data/" + makam + "/" + name) == True):
                    y.append(makams.index(makam))
                    break
            
            with open(os.path.join(root, name)) as f:
                content = f.read()
                all_qpitch.append(content)
print("File number:", len(y))
print("Array length:", len(all_qpitch))

File number: 1000
Array length: 1000


### Preprocessing
Pading input sequences

In [3]:
#Tokenization
top_k = 636 #53*4*3
tokenizer = Tokenizer(num_words=top_k, split='\n')
tokenizer.fit_on_texts(all_qpitch)
seqs = tokenizer.texts_to_sequences(all_qpitch)
print(seqs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
#Padding
X = sequence.pad_sequences(seqs, maxlen = 10000, padding='post')
print(X)
print(len(X))

[[ 1 22  8 ... 34  6  1]
 [ 1 60  9 ... 51  4  1]
 [ 1 28  3 ... 38  9  1]
 ...
 [ 2 12  8 ... 38  9  1]
 [ 1 20  5 ... 47  4  2]
 [ 1 47  4 ... 38  9  2]]
1000


### Library importing for deep learning

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

### Train - Test split

In [6]:
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, np.asarray(y), test_size=0.33, random_state=42)

In [7]:
print(y_train.shape)

(670, 20)


### Building the LSTM

In [11]:
#Word embedding
embedding_vector_len = 8 #max pitch value length: 4, comma, significance value of length 1, newline 

#Network topology
model = Sequential()

#model.add(Embedding(top_k, embedding_vector_len, input_length=len(X[0])))
model.add(Embedding(top_k, embedding_vector_len, input_length=10000)) #smaller input length for testing
model.add(Dropout(0.2))
model.add(LSTM(70))
model.add(Dropout(0.2))
model.add(Dense(20, activation='sigmoid'))

In [12]:
#Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10000, 8)          5088      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10000, 8)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 70)                22120     
_________________________________________________________________
dropout_4 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1420      
Total params: 28,628
Trainable params: 28,628
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Fitting
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
670/670 [==============================] - 127s 189ms/step - loss: 0.4521 - accuracy: 0.9001
Epoch 2/5
670/670 [==============================] - 128s 191ms/step - loss: 0.2239 - accuracy: 0.9499
Epoch 3/5
670/670 [==============================] - 124s 186ms/step - loss: 0.2102 - accuracy: 0.9500
Epoch 4/5
670/670 [==============================] - 124s 185ms/step - loss: 0.2067 - accuracy: 0.9500
Epoch 5/5
670/670 [==============================] - 123s 184ms/step - loss: 0.2049 - accuracy: 0.9500


In [14]:
#Model evaluation
scores = model.evaluate(X_test, y_test, verbose=1)
print(scores)

330/330 [==============================] - 9s 26ms/step
[0.19945193502036007, 0.9499999284744263]
